In [1]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [2]:
# set semantics and compute strength of arguments
filename = "../../bags/td_qbaf.bag"
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG(filename)
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 12.199999999999973
Argument(name=s0, weight=0.5, strength=0.664592759656642)
Argument(name=s1, weight=0.5, strength=0.664592759656642)
Argument(name=s2, weight=0.5, strength=0.664592759656642)
Argument(name=s3, weight=0.5, strength=0.8135424356888933)
Argument(name=s4, weight=0.5, strength=0.7041994642766211)
Argument(name=s5, weight=0.5, strength=0.7041994642766211)
Argument(name=s6, weight=0.5, strength=0.5000000446420562)
Argument(name=s7, weight=0.5, strength=0.8254237055377077)
Argument(name=s8, weight=0.5, strength=0.7235959804104234)
Argument(name=s9, weight=0.5, strength=0.7235959804104234)
Argument(name=s10, weight=0.5, strength=0.7235959804104234)
Argument(name=c0, weight=0.0, strength=0.7000991119110755)
Argument(name=c1, weight=0.0, strength=0.46753135307445565)
Argument(name=c2, weight=0.0, strength=0.8313656875032375)
Argument(name=c3, weight=0.0, strength=4.617127656482882e-07)
Argument(name=c4, weight=0.0, strength=3.570538632983414e-07)


0.0009878603392623568

In [3]:
# get all the attacks and supports in QBAF
att_relation = model.BAG.get_attacks()
sup_relation = model.BAG.get_supports()
all_relation = att_relation + sup_relation

In [4]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))
all_relation_new

[('att', 'c0', 'c1'),
 ('att', 'c1', 'c0'),
 ('att', 'c2', 'c3'),
 ('att', 'c3', 'c2'),
 ('att', 'c4', 'c5'),
 ('att', 'c5', 'c4'),
 ('sup', 's0', 'c0'),
 ('sup', 'c0', 's0'),
 ('sup', 's1', 'c0'),
 ('sup', 'c0', 's1'),
 ('sup', 's2', 'c0'),
 ('sup', 'c0', 's2'),
 ('sup', 's3', 'c1'),
 ('sup', 'c1', 's3'),
 ('sup', 's7', 'c1'),
 ('sup', 'c1', 's7'),
 ('sup', 's3', 'c2'),
 ('sup', 'c2', 's3'),
 ('sup', 's4', 'c2'),
 ('sup', 'c2', 's4'),
 ('sup', 's5', 'c2'),
 ('sup', 'c2', 's5'),
 ('sup', 's6', 'c3'),
 ('sup', 'c3', 's6'),
 ('sup', 's6', 'c4'),
 ('sup', 'c4', 's6'),
 ('sup', 's7', 'c5'),
 ('sup', 'c5', 's7'),
 ('sup', 's8', 'c5'),
 ('sup', 'c5', 's8'),
 ('sup', 's9', 'c5'),
 ('sup', 'c5', 's9'),
 ('sup', 's10', 'c5'),
 ('sup', 'c5', 's10')]

In [5]:
# set topic argument
topic_a = "c5"
topic_arg = model.BAG.arguments[topic_a]

In [6]:
# compute strength for all arguments
for arg in model.BAG.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('s0', 0.5, 0.664592759656642)
('s1', 0.5, 0.664592759656642)
('s2', 0.5, 0.664592759656642)
('s3', 0.5, 0.8135424356888933)
('s4', 0.5, 0.7041994642766211)
('s5', 0.5, 0.7041994642766211)
('s6', 0.5, 0.5000000446420562)
('s7', 0.5, 0.8254237055377077)
('s8', 0.5, 0.7235959804104234)
('s9', 0.5, 0.7235959804104234)
('s10', 0.5, 0.7235959804104234)
('c0', 0.0, 0.7000991119110755)
('c1', 0.0, 0.46753135307445565)
('c2', 0.0, 0.8313656875032375)
('c3', 0.0, 4.617127656482882e-07)
('c4', 0.0, 3.570538632983414e-07)
('c5', 0.0, 0.8996854332891212)


In [7]:
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations

for i in range(len(all_relation_new)):

    cur_relation = all_relation_new[i]


    # compute strength before removing
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
    topic_arg = model.BAG.arguments[topic_a]
    strength_before_drop_cur = topic_arg.strength


    # remove cur_relation
    if cur_relation[0]=="att":
        temp_attacker = model.BAG.remove_attack(cur_relation[1],cur_relation[2]) # cur_relation[1] attacks cur_relation[2]
    elif cur_relation[0]=="sup":
        temp_supporter = model.BAG.remove_support(cur_relation[1],cur_relation[2]) # cur_relation[1] supports cur_relation[2]


    # compute strength after removing
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
    topic_arg = model.BAG.arguments[topic_a]
    strength_after_drop_cur = topic_arg.strength

    # compute strength difference
    difference = strength_before_drop_cur - strength_after_drop_cur


    # compute attributions
    attribution[cur_relation] = difference

    # recover QBAF for the next iteration
    model = grad.semantics.QuadraticEnergyModel()
    model.approximator = grad.algorithms.RK4(model)
    model.BAG = grad.BAG(filename)
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 0.1
Argument(name=s0, weight=0.5, strength=0.6645795163103054)
Argument(name=s1, weight=0.5, strength=0.6645795163103054)
Argument(name=s2, weight=0.5, strength=0.6645795163103054)
Argument(name=s3, weight=0.5, strength=0.8135801418076822)
Argument(name=s4, weight=0.5, strength=0.704213732404874)
Argument(name=s5, weight=0.5, strength=0.704213732404874)
Argument(name=s6, weight=0.5, strength=0.5000000403938354)
Argument(name=s7, weight=0.5, strength=0.8254546770579669)
Argument(name=s8, weight=0.5, strength=0.7236031828193743)
Argument(name=s9, weight=0.5, strength=0.7236031828193743)
Argument(name=s10, weight=0.5, strength=0.7236031828193743)
Argument(name=c0, weight=0.0, strength=0.7000544029635203)
Argument(name=c1, weight=0.0, strength=0.46762655845034484)
Argument(name=c2, weight=0.0, strength=0.8313850974677484)
Argument(name=c3, weight=0.0, strength=4.1777502458728296e-07)
Argument(name=c4, weight=0.0, strength=3.23075725032213e-07)
Argument(name=

In [8]:
attribution

{('att', 'c0', 'c1'): -0.0025984734867826598,
 ('att', 'c1', 'c0'): 0.0006959664978454239,
 ('att', 'c2', 'c3'): -0.00011597661275386528,
 ('att', 'c3', 'c2'): -7.913231470424442e-06,
 ('att', 'c4', 'c5'): -7.914991274948768e-06,
 ('att', 'c5', 'c4'): 0.01470325176721976,
 ('sup', 's0', 'c0'): -0.0024360746511770115,
 ('sup', 'c0', 's0'): -0.0006410704965443825,
 ('sup', 's1', 'c0'): -0.0024360746511770115,
 ('sup', 'c0', 's1'): -0.0006410704965443825,
 ('sup', 's2', 'c0'): -0.0024360746511770115,
 ('sup', 'c0', 's2'): -0.0006410704965443825,
 ('sup', 's3', 'c1'): 0.006576019848735237,
 ('sup', 'c1', 's3'): 0.0008057792948840126,
 ('sup', 's7', 'c1'): 0.006577898123373616,
 ('sup', 'c1', 's7'): 0.006793937619424817,
 ('sup', 's3', 'c2'): 0.00021203602979047442,
 ('sup', 'c2', 's3'): 0.002946488050734941,
 ('sup', 's4', 'c2'): 8.519067152368276e-05,
 ('sup', 'c2', 's4'): -0.00012035049988745605,
 ('sup', 's5', 'c2'): 8.519067152368276e-05,
 ('sup', 'c2', 's5'): -0.00012035049988745605,
